# Active work in LIFE Target Database

# database creation

In [2]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

#provider_tables_dict, database_tables= ltd.partial_create(30,create=['life'])

provider_tables_dict, database_tables= ltd.create_life_td(30)

#issue: does not wokr with loading as then id_source_idref already exists -> put in a workaround with a warning. though now did not happen...

#issue: create not working right now because exomercat server not available -> put in a workaround
#issue: main_id missing in star_basic -> suddenly working??

Building life_td data with distance cut of 30 pc
Trying to create SIMBAD tables from 2025-02-14...


 sorting object types...
removing 22  objects that had object types: ['BD?|MIR', 'BD?|MIR|NIR', 'BD?|MIR|NIR|Opt', 'BD?|NIR']
example object of them: WISEA J153429.75-104303.3


Trying to create Grant Kennedy Disks tables from 2024-02-09...
Trying to create WDS tables from 2025-02-14...
Creating  WDS  tables ...
 querying VizieR for WDS...
 performing distance cut...


Trying to create Exo-MerCat tables from 2025-02-14...


Trying to create LIFE tables from 2025-02-14...
            main_id             sptype_string class_lum class_temp class_temp_nr
------------------------------- ------------- --------- ---------- -------------
        2MASS J05171315-7706040             M         V          M             0
                      BPM 44589             M         V          M             0
                    CD-34  1169             M         V          M             0
                    CD-81   173             M         V          M             0
Cl* Melotte   22     MT     111            M:         V          M             0
                       G 111-52             M         V          M             0
                       G 242-59             K         V          K             0
                      G 273-171            MV         V          M             0
                      HD 170209         K(2)V         V          K             0
                      L   60-13             M         V      

Building sources table ...
Building objects table ...


Building provider table ...
Building ident table ...


Building best_h_link table ...
Building star_basic table ...
Building planet_basic table ...
Building disk_basic table ...
Building mes_mass_pl table ...
Building mes_teff_st table ...
Building mes_radius_st table ...
Building mes_mass_st table ...
Building mes_binary table ...
Building mes_sep_ang table ...
Building h_link table ...


Unifying null values...
Saving data...


In [ ]:
from time import time,ctime

print(ctime(time()))

#most likely can gain performance by differentiating between single object and multiple object groups

#esier method for performance assertment is to print time at start and end of function

#to improve performance

import profile
profile.run('main()')

# load and analyze database

In [48]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

provider_tables_dict, database_tables= ltd.load_life_td()


Loading life_td generated data


Finding issue that causes M0-M5 stars to have too high temperatures (4250 instead of 3000)

In [42]:
print(database_tables)#from create is a dict, from load is a list

{'sources': <Table length=102>
                       ref                            provider_name    source_id
                      object                              object         int64  
-------------------------------------------------- ------------------- ---------
                               1967AJ.....72.1334C              SIMBAD         1
                               1980AJ.....85..454H              SIMBAD         2
                               1982A&A...108..406K              SIMBAD         3
                               1982ApJ...253..237C              SIMBAD         4
                               1982MSS...C03....0H              SIMBAD         5
                               1983SAAOC...7..106W              SIMBAD         6
                               1984ApJ...276..602G              SIMBAD         7
                               1985ApJS...59..197B              SIMBAD         8
                               1988AJ.....95.1841G              SIMBAD        

In [53]:
import numpy as np
#9=mesteff
#5=starbasic
#print(database_tables[5].colnames)
cols=['main_id','sptype_string','class_temp_nr','class_temp','teff_st_value','teff_st_source_idref']
mstars=database_tables['star_basic'][cols][np.where(database_tables['star_basic']['class_temp']=='M')]
mstars_tempgiven=mstars[np.where(mstars['teff_st_value']!=1e+20)]
print(mstars_tempgiven[np.where(mstars_tempgiven['teff_st_value']>4000)])

 main_id   sptype_string class_temp_nr ... teff_st_value teff_st_source_idref
                                       ...       K                           
---------- ------------- ------------- ... ------------- --------------------
 L  372-58         M5.5V           5.5 ...        4250.0                   93
LP  731-58         M6.5V           6.5 ...        4250.0                   93
 Ross  248         M5.0V           5.0 ...        4250.0                   93
 V* YZ Cet        M4.0Ve           4.0 ...        4250.0                   93


In [52]:
print(database_tables['sources'][np.where(database_tables['sources']['source_id']==93)])

            ref              provider_name source_id
---------------------------- ------------- ---------
2022arXiv220800211G GSP-Spec          Gaia        93


In [54]:
#so lets find out if in gaia archive this is also the case
#first find gaia id
print(database_tables['ident'][np.where(database_tables['ident']['main_id']=='L  372-58')])
#Gaia DR3 4848140361962951552

#on archive only gspphot showable and it is empty

             id               main_id  id_source_idref object_idref
---------------------------- --------- --------------- ------------
     2MASS J03355969-4430453 L  372-58              19          149
            APMPM J0336-4431 L  372-58              19          149
                    CNS5 919 L  372-58              19          149
                  GCRV 26222 L  372-58              19          149
            GEN# +9.85995046 L  372-58              19          149
                     GJ 1061 L  372-58              19          149
Gaia DR2 4848140361962951552 L  372-58              19          149
Gaia DR3 4848140361962951552 L  372-58              19          149
                   L  372-58 L  372-58              18          149
                   L  372-58 L  372-58              19          149
                    LFT  295 L  372-58              19          149
                   LHS  1565 L  372-58              19          149
                  LP  995-46 L  372-58          

In [60]:
from sdata import empty_dict
from provider.utils import fetch_main_id, IdentifierCreator, fill_sources_table, create_sources_table, ids_from_ident, replace_value, create_provider_table, query
from pyvo.dal import TAPService

adql_query="""
    SELECT top 10 s.source_id ,p.mass_flame, p.radius_flame,
        p.teff_gspphot, p.teff_gspspec, m.nss_solution_type, p.age_flame
    FROM gaiadr3.gaia_source as s
        JOIN gaiadr3.astrophysical_parameters as p ON s.source_id=p.source_id
            LEFT JOIN gaiadr3.nss_two_body_orbit as m ON s.source_id=m.source_id
            WHERE s.source_id = 4848140361962951552"""
gaia = empty_dict.copy()
gaia['provider'] = create_provider_table('Gaia',
                                  "https://gea.esac.esa.int/tap-server/tap",
                                  '2016A&A...595A...1G')
try: 
    gaia_helptab=query(gaia['provider']['provider_url'][0],adql_query) 

except:
    #because of bug in gaia server where async not working currently
    service = TAPService(gaia['provider']['provider_url'][0])
    result=service.run_sync(adql_query.format(**locals()), maxrec=160000)
    gaia_helptab=result.to_table()


Trying to create Gaia tables from 2025-02-10...


In [62]:
print(gaia_helptab['source_id','teff_gspspec'])

     source_id      teff_gspspec
                         K      
------------------- ------------
4848140361962951552       4250.0


In [ ]:
#so yes that is how it is in gaia. now find out if this parameter is reliable and if not lower its quality

In [ ]:
import analysis as lta

lta.sanity_tests(database_tables,5)